# Quant Finance - Clase 2

# Modulo yfinance para Yahoo Finance

Modulos necesarios

Codigo.data_nyse (en github) ;

pandas,

matplotlib,

datetime,

yfinance,

sys,

Instalarlos con Anaconda, comando Conda o comando pip install

In [ ]:
### Importo los modulos

import sys
sys.path.append('..')

from Codigo.data_nyse import *

import yfinance as yf
import datetime

## Queremos emular lo que hicimos para BYMA

In [ ]:
### Metodo para el web scraping
panel_yahoo_GGAL = obtener_opciones_yahoo_finance('GGAL')
panel_yahoo_GGAL


In [ ]:

panel_ociones_GGAL_nyse_clean = obtener_panel_opciones_nyse('GGAL', True)
panel_ociones_GGAL_nyse_clean

## Modulo yfinance para Yahoo Finance

In [ ]:
ticker = 'GGAL'
data = yf.Ticker(ticker)

#Data es un objeto
# data.info contiene informacion (mucha) del activo

data.info



In [ ]:
# Spot price

print('Bid: ', data.info['bid'])
print('Ask: ', data.info['ask'])
print('Avg: ', (data.info['bid']+data.info['ask'])/2)

In [ ]:
# Vencimientos de opciones
vencimientos = data.options

vencimientos

In [ ]:
# Calls y puts
calls = data.option_chain(vencimientos[0])[0]
puts = data.option_chain(vencimientos[0])[1]

puts

### Que hace internamente el metodo??

In [ ]:
#Recorro cada vencimiento disponible y voy tomando calls por un lado y puts por otro
for vencimiento in vencimientos[1:]:
        calls = calls.append(data.option_chain(vencimiento)[0])
        puts = puts.append(data.option_chain(vencimiento)[1])

#Uno los puts y los calls
panel_opciones = calls.append(puts)

# Le agrego dos columnas utiles - Ticker y Spot

panel_opciones['Spot'] = (data.info['bid']+data.info['ask'])/2
panel_opciones['Ticker'] = ticker

panel_opciones.head(10)

## Del panel de Yahoo crudo a otro con informacion customizada (igual que antes)

#### *) Agrego columnas utiles

In [ ]:

panel_opciones['Moneyness'] = 0.0
panel_opciones['TTM'] = 0
panel_opciones['CallPut'] = ''
panel_opciones = panel_opciones.reset_index()

panel_opciones.head(5)

#### *) Recorro las filas (1ra como ejemplo)

In [ ]:
idx = panel_opciones.index.values[0]

In [ ]:
callput = panel_opciones.contractSymbol.values[idx][10:11]
panel_opciones['CallPut'].values[idx] = callput

In [ ]:
year = 2000 + int(panel_opciones.contractSymbol.values[idx][4:6])
month = int(panel_opciones.contractSymbol.values[idx][6:8])
day = int(panel_opciones.contractSymbol.values[idx][8:10])

hoy = datetime.date.today()
expiry_datetime = datetime.date(year, month, day)
ttm = (expiry_datetime - hoy).days

panel_opciones['TTM'].values[idx] = ttm

In [ ]:
panel_opciones['Moneyness'].values[idx] = panel_opciones['Spot'].values[idx] / panel_opciones['strike'].values[idx]

In [ ]:
panel_opciones.head(2)

#### *) Recorro las filas (las restantes como ejemplo)

In [ ]:
for idx in list(panel_opciones.index.values[1:]):
        year = 2000 + int(panel_opciones.contractSymbol.values[idx][4:6])
        month = int(panel_opciones.contractSymbol.values[idx][6:8])
        day = int(panel_opciones.contractSymbol.values[idx][8:10])
        callput = panel_opciones.contractSymbol.values[idx][10:11]

        hoy = datetime.date.today()
        expiry_datetime = datetime.date(year, month, day)
        ttm = (expiry_datetime - hoy).days

        panel_opciones['TTM'].values[idx] = ttm
        panel_opciones['CallPut'].values[idx] = callput
        panel_opciones['Moneyness'].values[idx] = panel_opciones['Spot'].values[idx] / panel_opciones['strike'].values[idx]


In [ ]:
panel_opciones.head(10)

#### Panel Limpio

In [ ]:
panel_opciones = panel_opciones.rename(
            columns={'lastTradeDate': 'Last_Date', 'contractSymbol': 'Especie', 'strike': 'Strike', 'bid': 'Bid',
                     'ask': 'Ask', 'lastPrice': 'Last'}, inplace=False)

panel_opciones = panel_opciones.sort_values(['TTM', 'Ticker', 'Strike', 'CallPut'])

panel_opciones = panel_opciones[['Especie', 'Ticker', 'Spot', 'CallPut', 'Strike', 'TTM', 'Last', 'Moneyness']]


In [ ]:
panel_opciones

## Ploteando los datos

### Para cada vencimiento, grafico los strikes

In [ ]:
# NYSE
panel_opciones_ggal_nyse_limpio = obtener_panel_opciones_nyse('GGAL', True)

TTMs = list(set(panel_opciones_ggal_nyse_limpio.TTM.values))
TTMs

In [ ]:
TTM = 29
panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == 29) & (panel_opciones_ggal_nyse_limpio.CallPut == "C")].head(10)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs:
    Ks = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

CP = "P"
for TTM in TTMs:
    Ks = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()

### Para cada strike, grafico los TTM

In [ ]:
Ks = list(set(panel_opciones_ggal_nyse_limpio.Strike.values))

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for K in Ks:
    TTMs = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.Strike == K) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.Strike == K) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()
    

## Ploteando los datos con un ticker mega liquido

In [ ]:
# NYSE AAPL
ticker = 'AAPL'
panel_opciones_nyse_limpio = obtener_panel_opciones_nyse(ticker, True)

TTMs = list(set(panel_opciones_nyse_limpio.TTM.values))
TTMs.sort()
TTMs

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[0:5]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[5:11]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[11:17]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[0:5]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[5:11]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[11:17]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
Ks = list(set(panel_opciones_nyse_limpio.Strike.values))
Ks.sort()
len(Ks)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for K in Ks[45:55]:
    TTMs = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for K in Ks[100:]:
    TTMs = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al TTM')
    
plt.show()

## Pasar a Excel

In [ ]:
panel_opciones_ggal_nyse_crudo = obtener_panel_opciones_nyse('GGAL', False)
panel_opciones_ggal_nyse_crudo.to_excel('Clase_2_panel_opciones_ggal_nyse_crudo.xls')

panel_opciones_ggal_nyse_limpio = obtener_panel_opciones_nyse('GGAL', True)
panel_opciones_ggal_nyse_limpio.to_excel('Clase_2_panel_opciones_ggal_nyse_limpio.xls')


panel_opciones_aapl_nyse_limpio = obtener_panel_opciones_nyse('AAPL', True)
panel_opciones_aapl_nyse_limpio.to_excel('Clase_2_panel_opciones_aapl_nyse_limpio.xls')                                                
         